[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/agents/smolagents_hf_with_mongodb.ipynb)

# Using Smolagents with MongoDB Atlas

This notebook demonstrates how to use [Smolagents](https://github.com/huggingface/smolagents) to interact with MongoDB Atlas for building AI-powered applications. We'll explore how to create tools that leverage MongoDB's aggregation capabilities to analyze and extract insights from data.

## Prerequisites

Before running this notebook, you'll need:

1. A MongoDB Atlas account and cluster
2. Python environment with required packages
3. OpenAI API key for GPT-4 access

## Setting Up MongoDB Atlas

1. Create a free MongoDB Atlas account at [https://www.mongodb.com/cloud/atlas/register](https://www.mongodb.com/cloud/atlas/register)
2. Create a new cluster (free tier is sufficient)
3. Configure network access by adding your IP address (for google colab open `0.0.0.0/0` to test)
4. Create a database user with read/write permissions
5. Get your connection string from Atlas UI (Click "Connect" > "Connect your application")
6. Replace `<password>` in the connection string with your database user's password

## Observations

In this notebook, we:
- Define tools that interact with MongoDB Atlas using pymongo
- Use aggregation pipelines to analyze data
- Sample documents to understand schema structure
- Demonstrate how LLMs can generate and execute MongoDB queries

The tools showcase how to:
1. Execute aggregation pipelines generated by the LLM
2. Sample documents to understand collection structure
3. Handle errors and provide meaningful feedback

### Security Considerations

When working with MongoDB Atlas:
- Never commit connection strings with credentials to version control
- Use environment variables or secure secret management
- Restrict database user permissions to only what's needed
- Enable IP allowlist in Atlas Network Access settings

## Install dependencies

In [1]:
pip install pymongo smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 

### Place your connection string from MongoDB Atlas

In [2]:
import getpass
import os

MONGODB_URI = getpass.getpass("Enter your MongoDB Atlas URI: ")
os.environ["MONGODB_URI"] = MONGODB_URI

Enter your MongoDB Atlas URI: ··········


## Loading the dataset

In this example I am using the airbnb data set from https://huggingface.co/datasets/MongoDB/airbnb_embeddings .

- Database : ai_airbnb
- Collection : rentals

## Defining the tools

We'll create two main tools for interacting with MongoDB:

1. **Aggregation Tool**: Executes aggregation pipelines generated by the LLM to analyze data
   - Takes a pipeline as input
   - Handles complex data transformations
   - Returns aggregated results

2. **Sampling Tool**: Helps understand collection structure
   - Randomly samples documents
   - Provides schema insights
   - Useful for data exploration

Both tools automatically exclude embedding fields to reduce response size and improve readability.

In [3]:
from smolagents.agents import ToolCallingAgent, CodeAgent
from smolagents import tool, HfApiModel, TransformersModel, LiteLLMModel
from typing import Optional
from pymongo import MongoClient
from google.colab import userdata
import os
import getpass
import json

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

model = LiteLLMModel(model_id="gpt-4o")

client = MongoClient(MONGODB_URI, appname="devrel.showcase.smolagents")

@tool
def get_aggregated_docs(pipeline: str) -> list:
    """
    Gets a generated pipeline as 'pipeline' by the LLM and provide the context documents

     Args:
        pipeline: An array List with the current stages from the LLM # Added (list) and a description after the argument name
    """
    db = client["ai_airbnb"]
    collection = db["rentals"]
    pipeline = json.loads(pipeline)
    pipeline.insert(0, {"$project" : {"text_embeddings" : 0, "image_embeddings" : 0}}) # Use insert to add at the beginning
    docs = list(collection.aggregate(pipeline))
    return docs

@tool
def sample_documents(collection_name: str) -> str:
    """
    Use $sample to sample the collection docs

    Args:
      collection_name: The name of the collection to sample from
    """
    db = client["ai_airbnb"]
    try:
        collection = db[collection_name]
        sample = list(collection.aggregate([{"$project" : {"text_embeddings" : 0, "image_embeddings" : 0}},{"$sample": {"size": 5}}])) # Sample 5 documents
        return sample
    except Exception as e:
        return f"Error: {e}"

agent = ToolCallingAgent(tools=[get_aggregated_docs, sample_documents], model=model)

# Example usage
user_query = "What are the supported countries in our 'rentals' collection? Sample for structre and then  aggregate how many are in each country"
response = agent.run(user_query)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the supported countries in our 'rentals' collection, sample for structre and then  aggregate how many  │
│ are in each country                                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sample_documents' with arguments: {'collection_name': 'rentals'}                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: [{'_id': 7780335, 'listing_url': 'https://www.airbnb.com/rooms/7780335', 'name': 'Sunny studio in 
Williamsburg', 'summary': 'This south-facing, light-filled studio is a 3 minute walk from the Bedford L stop, right
in the heart of Williamsburg! Our building has a gym and roof with fantastic views of the Manhattan skyline.', 
'space': '', 'description': 'This south-facing, light-filled studio is a 3 minute walk from the Bedford L stop, 
right in the heart of Williamsburg! Our building has a gym and roof with fantastic views of the Manhattan 
skyline.', 'neighborhood_overview': '', 'notes': '', 'transit': '', 'access': '', 'interaction': '', 'house_rules':
'', 'property_type': 'Apartment', 'room_type': 'Entire home/apt', 'bed_type': 'Real Bed', 'minimum_nights': 3, 
'maximum_nights': 112, 'cancellation_policy': 'moderate', 'last_scraped': datetime.datetime(2019, 3, 6, 5, 0), 
'calendar_last_scraped': datetime.datetime(2019, 3, 6, 5, 0), 'first_review': datetime.datetime(2015, 9, 26, 4, 0),
'last_review': datetime.datetime(2017, 12, 29, 5, 0), 'accommodates': 2, 'bedrooms': 0.0, 'beds': 1.0, 
'number_of_reviews': 9, 'bathrooms': 1.0, 'amenities': ['TV', 'Cable TV', 'Internet', 'Wifi', 'Air conditioning', 
'Kitchen', 'Gym', 'Elevator', 'Buzzer/wireless intercom', 'Heating', 'Washer', 'Dryer', 'Smoke detector', 
'Essentials', 'Shampoo', 'Hair dryer', 'Laptop friendly workspace'], 'price': 150, 'security_deposit': None, 
'cleaning_fee': 50.0, 'extra_people': 0, 'guests_included': 1, 'images': {'thumbnail_url': '', 'medium_url': '', 
'picture_url': 'https://a0.muscache.com/im/pictures/99513018/97b35d0c_original.jpg?aki_policy=large', 
'xl_picture_url': ''}, 'host': {'host_id': '7285322', 'host_url': 'https://www.airbnb.com/users/show/7285322', 
'host_name': 'Allie', 'host_location': 'New York, New York, United States', 'host_about': '', 'host_response_time':
None, 'host_thumbnail_url': 
'https://a0.muscache.com/im/pictures/user/8b223da3-0868-40e8-b4fc-610b76abeee2.jpg?aki_policy=profile_small', 
'host_picture_url': 
'https://a0.muscache.com/im/pictures/user/8b223da3-0868-40e8-b4fc-610b76abeee2.jpg?aki_policy=profile_x_medium', 
'host_neighbourhood': 'Williamsburg', 'host_response_rate': None, 'host_is_superhost': False, 
'host_has_profile_pic': True, 'host_identity_verified': True, 'host_listings_count': 1, 
'host_total_listings_count': 1, 'host_verifications': ['email', 'phone', 'reviews', 'jumio', 'government_id']}, 
'address': {'street': 'Brooklyn, NY, United States', 'suburb': 'Williamsburg', 'government_area': 'Williamsburg', 
'market': 'New York', 'country': 'United States', 'country_code': 'US', 'location': {'type': 'Point', 
'coordinates': [-73.95352, 40.71607], 'is_location_exact': True}}, 'availability': {'availability_30': 0, 
'availability_60': 0, 'availability_90': 0, 'availability_365': 0}, 'review_scores': {'review_scores_accuracy': 10,
'review_scores_cleanliness': 10, 'review_scores_checkin': 10, 'review_scores_communication': 10, 
'review_scores_location': 10, 'review_scores_value': 9, 'review_scores_rating': 93}, 'reviews': [{'_id': 
'48403239', 'date': datetime.datetime(2015, 9, 26, 4, 0), 'listing_id': '7780335', 'reviewer_id': '40415432', 
'reviewer_name': 'Alain', 'comments': 'Studio confortable et superbement situé dans Williamsburg entre les stations
Bedford Av et Métropolitan, accès direct à Manhattan ou Brooklyn, Quartier vivant et sympathique, appartement calme
sur rue calme, \r\nExcellent contact avec notre hôte par plusieurs échanges courriel. \r\nCe logement agréable a 
contribué à ce que notre semaine à NYC soit parfaite!'}, {'_id': '74396139', 'date': datetime.datetime(2016, 5, 15,
4, 0), 'listing_id': '7780335', 'reviewer_id': '2437276', 'reviewer_name': 'Nicolas', 'comments': 'The Studio was 
perfect for a NY trip, nice location, just 5 minutes away from the Subway. Clean, quiet and spacious. '}, {'_id': 
'76744858', 'date': datetime.datetime(2016, 5, 29, 4, 0), 'listing_id': '7780335',

...

[Step 1: Duration 3.01 seconds| Input tokens: 39,110 | Output tokens: 57]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "The supported countries in the 'rentals' collection    │
│ and the number of rentals in each country are as follows:\n\n- United States: 1,222 rentals\n- Canada: 649      │
│ rentals\n- Turkey: 661 rentals\n- Australia: 610 rentals\n- Hong Kong: 600 rentals\n- Spain: 633 rentals\n-     │
│ Brazil: 606 rentals\n- Portugal: 555 rentals\n- China: 19 rentals"}                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The supported countries in the 'rentals' collection and the number of rentals in each country are as 
follows:

- United States: 1,222 rentals
- Canada: 649 rentals
- Turkey: 661 rentals
- Australia: 610 rentals
- Hong Kong: 600 rentals
- Spain: 633 rentals
- Brazil: 606 rentals
- Portugal: 555 rentals
- China: 19 rentals

[Step 2: Duration 2.12 seconds| Input tokens: 77,312 | Output tokens: 160]



## Conclusions

This notebook successfully demonstrates the integration of Smolagents with MongoDB Atlas, enabling effective data analysis through an AI agent.  The defined tools, `get_aggregated_docs` and `sample_documents`, effectively interact with the Airbnb dataset stored in MongoDB Atlas.  The agent, powered by a chosen LLM (in this case, GPT-4o), successfully translates user queries into both data sampling and aggregation pipelines executed against the MongoDB database.

Key improvements and observations include:

* **Robust Tool Design:** The tools now incorporate error handling, providing more informative feedback to the user in case of issues.  The exclusion of embedding fields from queries enhances performance and readability of results.
* **Enhanced Query Handling:**  The inclusion of an initial projection stage in the aggregation pipeline, specifically designed to remove embedding fields (`text_embeddings` and `image_embeddings`) prior to other stages, ensures more efficient query execution and smaller response sizes.  The use of `json.loads()` ensures that the pipeline string received from the LLM is correctly parsed.
* **Improved User Experience:** Clearer tool documentation and example usage further enhance the user's ability to interact with the agent and interpret results.
* **Practical Application:** The demonstration showcases a practical application for analyzing data within a MongoDB Atlas database using an LLM-powered agent.

Future development could explore:

* **Expanded Toolset:** Implementing additional tools for data manipulation, filtering, and more complex analytics.
* **Advanced Query Generation:** Exploring methods to refine the LLM's ability to generate accurate and efficient MongoDB queries.
* **Visualization Capabilities:** Integrating data visualization libraries to present the analysis results more effectively.
* **Security Enhancements:** Further solidifying security practices, potentially incorporating environment variable management for sensitive credentials.